In [57]:
import pandas as pd
url = "https://desafiogrupo1.s3.amazonaws.com/phishing.json"
df = pd.read_json(url)

In [79]:
df.head()

,_id,domain_name,category,dns,evaluated_on,ip_data,label,rdap,remarks,source,sourced_on,tls,url
0,{'$oid': '662ef11c9c88568844e38f1a'},conflrma57843.com,phishing,"{'A': ['188.114.97.9', '188.114.96.9'], 'AAAA'...",{'$date': '2024-04-29T03:19:57.352Z'},"[{'ip': '188.114.97.9', 'from_record': 'A', 'r...",phishing_since_2402,"{'handle': '2876141922_DOMAIN_COM-VRSN', 'name...",{'dns_evaluated_on': {'$date': '2024-04-29T03:...,openphish,{'$date': '2024-04-29T03:00:12.351Z'},"{'cipher': 'TLS_AES_256_GCM_SHA384', 'count': ...",https://conflrma57843.com/u6383644457
1,{'$oid': '660caa1a528344e2948e0c13'},www.zgahdjm.com,phishing,"{'A': None, 'AAAA': None, 'CNAME': None, 'MX':...",{'$date': '2024-04-03T03:14:49.179Z'},None,phishing_since_2402,"{'handle': '', 'name': 'zgahdjm.com', 'whois_s...",{'dns_evaluated_on': {'$date': '2024-04-03T03:...,phishtank,{'$date': '2024-04-03T03:00:08.447Z'},"{'cipher': 'TLS_AES_256_GCM_SHA384', 'count': ...",https://www.zgahdjm.com
2,{'$oid': '65c4d948ec20407e161504ca'},attt.pages.dev,phishing,"{'A': ['172.66.44.147', '172.66.47.109'], 'AAA...",{'$date': '2024-01-30T11:02:40.844Z'},"[{'ip': '2606:4700:310c::ac42:2f6d', 'from_rec...",phishing_since_2402,"{'handle': '', 'name': 'pages.dev', 'whois_ser...",{'dns_evaluated_on': {'$date': '2024-01-30T11:...,phishtank,{'$date': '2024-01-30T09:31:01.663Z'},"{'cipher': 'TLS_AES_256_GCM_SHA384', 'count': ...",https://attt.pages.dev/
3,{'$oid': '65d75308bdfee21c31b781fa'},chermate.weeblysite.com,phishing,"{'A': ['162.159.140.60', '172.66.0.60'], 'AAAA...",{'$date': '2023-11-20T18:17:45.439Z'},"[{'ip': '172.66.0.60', 'from_record': 'A', 're...",phishing_since_2402,"{'handle': '', 'name': 'weeblysite.com', 'whoi...",{'dns_evaluated_on': {'$date': '2023-11-20T18:...,phishtank,{'$date': '2023-11-20T17:36:44.311Z'},"{'cipher': 'TLS_AES_256_GCM_SHA384', 'count': ...",https://chermate.weeblysite.com/
4,{'$oid': '660f4d1a528344e29492a240'},allegrolokalnie.oferta73.pl,phishing,"{'A': ['84.32.84.33'], 'AAAA': None, 'CNAME': ...",{'$date': '2024-04-05T03:12:20.41Z'},"[{'ip': '84.32.84.33', 'from_record': 'A', 're...",phishing_since_2402,"{'handle': '', 'name': 'oferta73.pl', 'whois_s...",{'dns_evaluated_on': {'$date': '2024-04-05T03:...,phishtank,{'$date': '2024-04-05T03:00:08.815Z'},None,https://allegrolokalnie.oferta73.pl/oferta/l/i...


In [62]:
df.shape

(164425, 13)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164425 entries, 0 to 164424
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   _id           164425 non-null  object
 1   domain_name   164425 non-null  object
 2   category      164425 non-null  object
 3   dns           164425 non-null  object
 4   evaluated_on  164425 non-null  object
 5   ip_data       134910 non-null  object
 6   label         164425 non-null  object
 7   rdap          150816 non-null  object
 8   remarks       164425 non-null  object
 9   source        164425 non-null  object
 10  sourced_on    164425 non-null  object
 11  tls           131129 non-null  object
 12  url           164425 non-null  object
dtypes: object(13)
memory usage: 16.3+ MB


In [77]:
pd.reset_option('display.max_colwidth')

In [21]:
pd.set_option('display.max_colwidth', None)

In [110]:
df_phishing = pd.DataFrame()

In [111]:
df_phishing['DomainLength'] = df['domain_name'].str.len()

In [112]:
df_phishing['URL_Length'] = df['url'].str.len()

In [113]:
import ipaddress

def is_ip(value):
    try:
        ipaddress.ip_address(str(value))
        return 1   # es IP
    except ValueError:
        return 0   # es dominio

df_phishing["IsDomainIP"] = df["domain_name"].apply(is_ip)

In [114]:
%pip install tldextract

Note: you may need to restart the kernel to use updated packages.


In [115]:
import tldextract

# Extraer solo la parte de subdomain
df_phishing["subdomain"] = df["domain_name"].apply(lambda x: tldextract.extract(x).subdomain)

# Contar cuantos subdomains tiene
df_phishing["SubdomainCount"] = df_phishing["subdomain"].apply(lambda x: len(x.split(".")) if x else 0)

# ¿más de 3 subdomains?
df_phishing["HasMoreThan3Subdomains"] = df_phishing["SubdomainCount"] > 3

In [116]:
df_phishing.drop(columns=['subdomain'], inplace=True)

In [ ]:
import re

def obfuscation_ratio(s: str) -> float:
    if not isinstance(s, str) or len(s) == 0:
        return 0.0
    # Contar caracteres que no son letras (a-z)
    obfus = sum(1 for c in s.lower() if not re.match(r"[a-z]", c))
    return obfus / len(s)

df_phishing["ObfuscationRatio"] = df["domain_name"].apply(obfuscation_ratio).round(3)
df_phishing["HasObfuscation"] = (df_phishing["ObfuscationRatio"] > 0.2).astype(int)

In [122]:
def char_continuation_rate(s: str) -> float:
    if not isinstance(s, str) or len(s) == 0:
        return 0.0
    total = len(s)
    repeats = 0
    # Contar caracteres iguales al anterior
    for i in range(1, total):
        if s[i] == s[i-1]:
            repeats += 1
    return repeats / total

df_phishing["CharContinuationRate"] = df["domain_name"].apply(char_continuation_rate)
df_phishing["HasCharContinuation"] = (df_phishing["CharContinuationRate"] > 0.7).astype(int)

In [ ]:
import string

# Distribución esperada (muy simplificada)
char_probs = {}
for c in string.ascii_lowercase:  # letras
    char_probs[c] = 0.08
for c in string.digits:           # números
    char_probs[c] = 0.02

# símbolos comunes en URLs
char_probs.update({
    "-": 0.02, 
    ".": 0.05,
    "/": 0.05,
    "?": 0.02,
    "_": 0.01,
    "&": 0.01,
    "=": 0.01,
    "#": 0.01,
    ":": 0.03,
    "~": 0.005,
    "+": 0.005
})

# cualquier otro carácter
default_prob = 0.001

def url_char_prob(url: str) -> float:
    if not isinstance(url, str) or len(url) == 0:
        return 0.0
    probs = []
    for c in url.lower():
        probs.append(char_probs.get(c, default_prob))
    return sum(probs) / len(probs)

# Aplicar al DataFrame
df_phishing["URLCharProb"] = df["url"].apply(url_char_prob)
df_phishing["HasCharAnomaly"] = (df_phishing["URLCharProb"] < 0.1).astype(int)


In [141]:
df_phishing

,DomainLength,URL_Length,IsDomainIP,SubdomainCount,HasMoreThan3Subdomains,ObfuscationRatio,HasObfuscation,CharContinuationRate,HasCharContinuation,URLCharProb,HasCharAnomaly
0,17,37,0,0,False,0.353,1,0.000000,0,0.051081,1
1,15,23,0,1,False,0.133,0,0.133333,0,0.072609,1
2,14,23,0,1,False,0.143,0,0.142857,0,0.071304,1
3,23,32,0,1,False,0.087,0,0.043478,0,0.073750,1
4,27,111,0,1,False,0.148,0,0.037037,0,0.053072,1
...,...,...,...,...,...,...,...,...,...,...,...
164420,12,27,0,0,False,0.167,0,0.083333,0,0.069259,1
164421,43,104,0,1,False,0.442,1,0.046512,0,0.055577,1
164422,21,57,0,2,False,0.381,1,0.047619,0,0.058474,1
164423,41,50,0,1,False,0.098,0,0.024390,0,0.073600,1


In [142]:
df_phishing.HasCharAnomaly.value_counts()

HasCharAnomaly
1    164425
Name: count, dtype: int64